   # Pytorch Convolutional Neural Network
   This tutorial is based on https://github.com/yunjey/pytorch-tutorial

In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'GPU: {torch.cuda.is_available()}')

GPU: True


In [8]:
PATH=f'/tmp/'
# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

## Define the Data

In [9]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root=f'{PATH}/data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root=f'{PATH}/data', 
                                          train=False, 
                                          transform=transforms.ToTensor())
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


## Create a Model

In [10]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)
model

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

## Define loss function and optimizer

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Start the training
    - call model (model(images))
    - calculate loss (criterion(outputs,labels)
    - backprop (optimizer.zero_grad(), loss.backward())
    - apply grad (optimizer.step())

In [12]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1891
Epoch [1/5], Step [200/600], Loss: 0.1264
Epoch [1/5], Step [300/600], Loss: 0.0738
Epoch [1/5], Step [400/600], Loss: 0.0782
Epoch [1/5], Step [500/600], Loss: 0.0422
Epoch [1/5], Step [600/600], Loss: 0.1796
Epoch [2/5], Step [100/600], Loss: 0.1624
Epoch [2/5], Step [200/600], Loss: 0.0526
Epoch [2/5], Step [300/600], Loss: 0.0272
Epoch [2/5], Step [400/600], Loss: 0.0039
Epoch [2/5], Step [500/600], Loss: 0.0274
Epoch [2/5], Step [600/600], Loss: 0.0252
Epoch [3/5], Step [100/600], Loss: 0.0070
Epoch [3/5], Step [200/600], Loss: 0.0125
Epoch [3/5], Step [300/600], Loss: 0.0573
Epoch [3/5], Step [400/600], Loss: 0.0032
Epoch [3/5], Step [500/600], Loss: 0.0284
Epoch [3/5], Step [600/600], Loss: 0.0131
Epoch [4/5], Step [100/600], Loss: 0.0069
Epoch [4/5], Step [200/600], Loss: 0.0153
Epoch [4/5], Step [300/600], Loss: 0.0042
Epoch [4/5], Step [400/600], Loss: 0.0084
Epoch [4/5], Step [500/600], Loss: 0.0500
Epoch [4/5], Step [600/600], Loss:

## Test the model

In [13]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 99.17 %
